In [ ]:
"""
Created on Mon Jul 17 14:42 2023

This script is to plot all T and S profiles from the training ensemble

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import basal_melt_param.useful_functions as uf
import basal_melt_param.melt_functions as meltf
from basal_melt_param.constants import *
import basal_melt_param.T_S_profile_functions as tspf
import seaborn as sns
from matplotlib import cm
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import cmocean
import glob

In [ ]:
%matplotlib qt5

READ IN DATA

In [ ]:
plot_path = '/bettik/burgardc/PLOTS/summer_paper/'
outputpath_simple_all = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/SIMPLE/nemo_5km_06161821/'


In [ ]:
run_list = ['OPM031','OPM026','OPM021','OPM018','OPM016','OPM006'] #,'isf94','isfru94'] #,'OPM026','OPM027','OPM031'] 
#run_list = ['OPM021'] 
file_isf_list = []
TS_profile_mean_list = []
TS_profile_std_list = []
TS_profile_list = []
plume_list = []

for n,nemo_run in enumerate(run_list):
    
    print(nemo_run)
    
    # File mask
    if nemo_run in ['ctrl94','isf94','isfru94']:
        inputpath_mask = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
        file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS.nc')
    else:
        inputpath_mask = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
        file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')
    nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
    file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
    large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
    file_isf = file_isf_nonnan.sel(Nisf=large_isf)
    file_isf_list.append(file_isf)
    
    if nemo_run in ['ctrl94','isf94','isfru94']:
        inputpath_profiles = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'
        file_TS_orig = xr.open_dataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_allyy.nc')
        file_TS = file_TS_orig.sel(Nisf=large_isf)
    else:
        inputpath_profiles = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'
        file_TS_orig = xr.open_dataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_and_offshore_1980-2018_oneFRIS.nc')
        if nemo_run == 'OPM006':
            file_TS_otherdepth = file_TS_orig.sel(Nisf=large_isf, profile_domain=50).squeeze().drop('profile_domain')
            file_TS = file_TS_otherdepth.interp({'depth': TS_profile_mean_list[0].depth})
        else:
            file_TS = file_TS_orig.sel(Nisf=large_isf, profile_domain=50).squeeze().drop('profile_domain')
    file_TS_mean = file_TS.mean('time')
    file_TS_std = file_TS.std('time')
    TS_profile_mean_list.append(file_TS_mean)
    TS_profile_std_list.append(file_TS_std)
    TS_profile_list.append(file_TS.assign_coords({'nemo_run': nemo_run}))
    
    if nemo_run in ['ctrl94','isf94','isfru94']:
        inputpath_plumes = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/PLUMES/nemo_5km_'+nemo_run+'/'
    else:
        inputpath_plumes = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/PLUMES/nemo_5km_'+nemo_run+'/'
    plume_charac = xr.open_dataset(inputpath_plumes+'nemo_5km_plume_characteristics_oneFRIS.nc')
    plume_list.append(plume_charac)
    
file_isf_all = xr.concat(file_isf_list, dim='nemo_run')

TS_profile_mean = xr.concat(TS_profile_mean_list, dim='nemo_run').assign_coords({'nemo_run': run_list})
TS_profile_std = xr.concat(TS_profile_std_list, dim='nemo_run').assign_coords({'nemo_run': run_list})
TS_profile_all = xr.concat(TS_profile_list, dim='nemo_run').assign_coords({'nemo_run': run_list})
plume_mathiot =  xr.concat(plume_list, dim='nemo_run').assign_coords({'nemo_run': run_list})

In [ ]:
run_list2 = ['ctrl94']#,'isf94','isfru94'] #,'OPM026','OPM027','OPM031'] 
#run_list = ['OPM021'] 
file_isf_list = []
TS_profile_mean_list = []
TS_profile_std_list = []
TS_profile_list = []
plume_list = []

for n,nemo_run in enumerate(run_list2):
    
    print(nemo_run)
    
    # File mask
    if nemo_run in ['ctrl94','isf94','isfru94']:
        inputpath_mask = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
        file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS.nc')
    else:
        inputpath_mask = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
        file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')
    nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
    file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
    large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
    file_isf = file_isf_nonnan.sel(Nisf=large_isf)
    file_isf_list.append(file_isf)
    
    if nemo_run in ['ctrl94','isf94','isfru94']:
        inputpath_profiles = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'
        file_TS_orig = xr.open_dataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_allyy.nc')
        file_TS = file_TS_orig.sel(Nisf=large_isf)
    else:
        inputpath_profiles = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'
        file_TS_orig = xr.open_dataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_and_offshore_1980-2018_oneFRIS.nc')
        if nemo_run == 'OPM006':
            file_TS_otherdepth = file_TS_orig.sel(Nisf=large_isf, profile_domain=50).squeeze().drop('profile_domain')
            file_TS = file_TS_otherdepth.interp({'depth': TS_profile_mean_list[0].depth})
        else:
            file_TS = file_TS_orig.sel(Nisf=large_isf, profile_domain=50).squeeze().drop('profile_domain')
    file_TS_mean = file_TS.mean('time')
    file_TS_std = file_TS.std('time')
    TS_profile_mean_list.append(file_TS_mean)
    TS_profile_std_list.append(file_TS_std)
    TS_profile_list.append(file_TS.assign_coords({'nemo_run': nemo_run}))
    
    if nemo_run in ['ctrl94','isf94','isfru94']:
        inputpath_plumes = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/PLUMES/nemo_5km_'+nemo_run+'/'
    else:
        inputpath_plumes = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/PLUMES/nemo_5km_'+nemo_run+'/'
    plume_charac = xr.open_dataset(inputpath_plumes+'nemo_5km_plume_characteristics_oneFRIS.nc')
    plume_list.append(plume_charac)
    
file_isf_all = xr.concat(file_isf_list, dim='nemo_run')

TS_profile_mean2 = xr.concat(TS_profile_mean_list, dim='nemo_run').assign_coords({'nemo_run': run_list2})
TS_profile_std2 = xr.concat(TS_profile_std_list, dim='nemo_run').assign_coords({'nemo_run': run_list2})
TS_profile_all2 = xr.concat(TS_profile_list, dim='nemo_run').assign_coords({'nemo_run': run_list2})
plume_christoph =  xr.concat(plume_list, dim='nemo_run').assign_coords({'nemo_run': run_list2})

In [ ]:
TS_profile_mean2['theta_ocean'].sel(Nisf=66,nemo_run='ctrl94').plot()

In [ ]:
file_TS['theta_ocean'].isel(time=0).sel(Nisf=66).plot()

In [ ]:
isf_dim = [10, 11, 12, 13, 18, 22, 23, 24, 25, 30, 31, 33, 38, 39, 40, 42, 43, 44, 45, 47, 48, 51, 52, 53, 54, 55, 58, 61, 65, 66, 69, 70, 71, 73, 75]
sorted_isf_all = [11,69,43,12,70,44,13,58,71,45,30,31,61,73,47,48,33,18,10,65,51,22,38,52,23,66,53,39,24,40,54,75,25,42,55]

In [ ]:
len(sorted_isf_all)

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','Eastern Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue']

reg_list = file_isf['region'].astype(str)

alpha_val = 0.07

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

leg_hdl = []

i = 0

for kisf in tqdm(sorted_isf_all):

    ax[i] = f.add_subplot(6,6,i+1)



    for n,nrun in enumerate(run_list):

        for tt in TS_profile_all.sel(nemo_run = nrun).time:

            T_data = TS_profile_all['theta_ocean'].sel(nemo_run = nrun, time = tt, Nisf = kisf)
            ax[i].plot(T_data,-1*T_data.depth, color=colors[n], alpha=alpha_val)
        
    for n,nrun in enumerate(run_list2):
        
        for tt in TS_profile_all2.sel(nemo_run = nrun).time:

            T_data2 = TS_profile_all2['theta_ocean'].sel(nemo_run = nrun, time = tt, Nisf = kisf)
            ax[i].plot(T_data2,-1*T_data2.depth, color=colors[n], alpha=alpha_val)


        #leg, = plt.plot(mean_run,-1*file_TS_mean_all.depth, color=colors[n], linewidth=3)
        #plt.fill_betweenx(-1*mean_run.depth, mean_run - std_run, mean_run + std_run, -1*mean_run.depth, alpha=0.2, color=colors[n])
        #leg_hdl.append(leg)

    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    ax[i].set_xlim(-2.2,4)

    i = i+1


f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
#f.savefig(plot_path + 'T_profiles_all.png', dpi=300)

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','Eastern Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue']

reg_list = file_isf['region'].astype(str)

alpha_val = 0.07

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

leg_hdl = []

i = 0

for kisf in tqdm(sorted_isf_all):

    ax[i] = f.add_subplot(6,6,i+1)



    for n,nrun in enumerate(run_list):

        for tt in TS_profile_all.sel(nemo_run = nrun).time:

            T_data = TS_profile_all['salinity_ocean'].sel(nemo_run = nrun, time = tt, Nisf = kisf)
            ax[i].plot(T_data,-1*T_data.depth, color=colors[n], alpha=alpha_val)
        
    for n,nrun in enumerate(run_list2):
        
        for tt in TS_profile_all2.sel(nemo_run = nrun).time:

            T_data2 = TS_profile_all2['salinity_ocean'].sel(nemo_run = nrun, time = tt, Nisf = kisf)
            ax[i].plot(T_data2,-1*T_data2.depth, color=colors[n], alpha=alpha_val)


        #leg, = plt.plot(mean_run,-1*file_TS_mean_all.depth, color=colors[n], linewidth=3)
        #plt.fill_betweenx(-1*mean_run.depth, mean_run - std_run, mean_run + std_run, -1*mean_run.depth, alpha=0.2, color=colors[n])
        #leg_hdl.append(leg)

    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    ax[i].set_xlim(31,35)

    i = i+1


f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
#f.savefig(plot_path + 'T_profiles_all.png', dpi=300)